# 导入module

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm

device = torch.device('cuda')

# 1. 自监督学习

## 1.1 对比学习

In [12]:
class SimCLRTransform:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __call__(self, x):
        return self.transform(x), self.transform(x)


batch_size = 256  
dataset = datasets.STL10(root='autodl-tmp/data/data', split='unlabeled', transform=SimCLRTransform(), download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# 定义ResNet-18模型
class ResNetSimCLR(nn.Module):
    def __init__(self, base_model, out_dim):
        super(ResNetSimCLR, self).__init__()
        self.encoder = nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Sequential(
            nn.Linear(base_model.fc.in_features, 2048),
            nn.ReLU(),
            nn.Linear(2048, out_dim)
        )

    def forward(self, x):
        h = self.encoder(x)
        h = h.view(h.size(0), -1)
        z = self.fc(h)
        return h, z


base_model = resnet18(weights=None)  
model = ResNetSimCLR(base_model, out_dim=128).to(device)


class NTXentLoss(nn.Module):
    def __init__(self, batch_size, temperature):
        super(NTXentLoss, self).__init__()
        self.batch_size = batch_size
        self.temperature = temperature
        self.mask = self.mask_correlated_samples(batch_size)
        self.criterion = nn.CrossEntropyLoss(reduction="sum")

    def mask_correlated_samples(self, batch_size):
        N = 2 * batch_size  
        mask = torch.ones((N, N), dtype=bool)  
        mask = mask.fill_diagonal_(0)  

        for i in range(batch_size):
            mask[i, batch_size + i] = 0  
            mask[batch_size + i, i] = 0  

        return mask

    def forward(self, z_i, z_j):
        batch_size = z_i.size(0) 
        N = 2 * batch_size
        z = torch.cat((z_i, z_j), dim=0)  
        sim = torch.matmul(z, z.T) / self.temperature  

        
        assert sim.shape == (N, N), f"Sim matrix shape is {sim.shape}, expected ({N}, {N})"

        sim_i_j = torch.diag(sim, batch_size)  
        sim_j_i = torch.diag(sim, -batch_size)

        
        assert sim_i_j.shape == (batch_size,), f"sim_i_j shape is {sim_i_j.shape}, expected ({batch_size},)"
        assert sim_j_i.shape == (batch_size,), f"sim_j_i shape is {sim_j_i.shape}, expected ({batch_size},)"

        positive_samples = torch.cat((sim_i_j, sim_j_i), dim=0).reshape(N, 1)
        mask = self.mask_correlated_samples(batch_size)  
        negative_samples = sim[mask].reshape(N, -1)

        
        assert negative_samples.shape == (N, N-2), f"negative_samples shape is {negative_samples.shape}, expected ({N}, {N-2})"

        labels = torch.zeros(N).to(positive_samples.device).long()
        logits = torch.cat((positive_samples, negative_samples), dim=1)
        loss = self.criterion(logits, labels)
        return loss / N


criterion = NTXentLoss(batch_size=batch_size, temperature=0.5).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3) 


def train_simclr(model, dataloader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
        for images, _ in progress_bar:
            images[0] = images[0].to(device)
            images[1] = images[1].to(device)
            optimizer.zero_grad()

            # 将批次中的每个图像增强两次
            z_i, _ = model(images[0])
            z_j, _ = model(images[1])

            # 计算对比损失
            loss = criterion(z_i, z_j)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())
        
        
        print(f"Epoch [{epoch+1}/{epochs}] completed.")
        print(f"Allocated memory: {torch.cuda.memory_allocated(device)} bytes")
        print(f"Reserved memory: {torch.cuda.memory_reserved(device)} bytes")
        
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(dataloader):.4f}')


train_simclr(model, dataloader, criterion, optimizer, epochs=50)
torch.save(model.state_dict(), 'resnet_simclr.pth')


Files already downloaded and verified


Epoch 1/50: 100%|██████████| 391/391 [03:30<00:00,  1.86it/s, loss=7.32] 


Epoch [1/50] completed.
Allocated memory: 1982695936 bytes
Reserved memory: 18452840448 bytes
Epoch [1/50], Loss: 17.0222


Epoch 2/50: 100%|██████████| 391/391 [04:21<00:00,  1.49it/s, loss=5.56]


Epoch [2/50] completed.
Allocated memory: 1982695936 bytes
Reserved memory: 18452840448 bytes
Epoch [2/50], Loss: 6.1618


...

Epoch [48/50] completed.
Allocated memory: 1982695936 bytes
Reserved memory: 18452840448 bytes
Epoch [48/50], Loss: 0.5230


Epoch 49/50: 100%|██████████| 391/391 [04:03<00:00,  1.60it/s, loss=0.397]


Epoch [49/50] completed.
Allocated memory: 1982695936 bytes
Reserved memory: 18452840448 bytes
Epoch [49/50], Loss: 0.5225


Epoch 50/50: 100%|██████████| 391/391 [03:56<00:00,  1.65it/s, loss=0.403]


Epoch [50/50] completed.
Allocated memory: 1982695936 bytes
Reserved memory: 18452840448 bytes
Epoch [50/50], Loss: 0.5097


## 1.2 监督学习训练输出层

In [14]:
# CIFAR-100 数据集加载
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.CIFAR100(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR100(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4)


class LinearClassifier(nn.Module):
    def __init__(self, encoder, num_classes):
        super(LinearClassifier, self).__init__()
        self.encoder = encoder
        self.fc = nn.Linear(512, num_classes)  # 512是ResNet-18编码器输出的特征维度

    def forward(self, x):
        with torch.no_grad():
            features = self.encoder(x)
            features = features.view(features.size(0), -1)
        out = self.fc(features)
        return out

# 加载预训练模型权重
model = ResNetSimCLR(resnet18(weights=None), out_dim=128).to(device)
model.load_state_dict(torch.load('resnet_simclr.pth'))

# 冻结编码器部分，只训练线性分类器
for param in model.encoder.parameters():
    param.requires_grad = False

linear_classifier = LinearClassifier(model.encoder, num_classes=100).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear_classifier.fc.parameters(), lr=1e-3)

# 训练线性分类器
def train_linear_classifier(model, dataloader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        correct = 0
        total = 0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            progress_bar.set_postfix(loss=loss.item(), accuracy=100.*correct/total)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(dataloader):.4f}, Accuracy: {100.*correct/total:.2f}%')

train_linear_classifier(linear_classifier, train_loader, criterion, optimizer, epochs=50)


Files already downloaded and verified
Files already downloaded and verified


Epoch 1/50: 100%|██████████| 196/196 [00:24<00:00,  8.11it/s, accuracy=21.8, loss=3.52]


Epoch [1/50], Loss: 4.0219, Accuracy: 21.77%


Epoch 2/50: 100%|██████████| 196/196 [00:23<00:00,  8.35it/s, accuracy=35, loss=3.02]  


Epoch [2/50], Loss: 3.2560, Accuracy: 35.00%


...

Epoch [48/50], Loss: 1.6710, Accuracy: 56.58%


Epoch 49/50: 100%|██████████| 196/196 [00:23<00:00,  8.23it/s, accuracy=56.9, loss=1.87]


Epoch [49/50], Loss: 1.6651, Accuracy: 56.92%


Epoch 50/50: 100%|██████████| 196/196 [00:23<00:00,  8.47it/s, accuracy=56.9, loss=1.65]

Epoch [50/50], Loss: 1.6601, Accuracy: 56.90%


# 2. 监督学习

## 2.1 加载预训练模型微调

In [16]:
# 加载在 ImageNet 上预训练的 ResNet-18 模型
pretrained_model = resnet18(pretrained=True).to(device)

# 冻结编码器部分，只训练线性分类器
for param in pretrained_model.parameters():
    param.requires_grad = False

# 替换全连接层，调整输出维度为 100
num_features = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_features, 100).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.fc.parameters(), lr=1e-3)



Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [02:58<00:00, 263kB/s]
Epoch 1/50: 100%|██████████| 196/196 [00:23<00:00,  8.23it/s, accuracy=37.5, loss=2.14]


Epoch [1/50], Loss: 2.8293, Accuracy: 37.46%


Epoch 2/50: 100%|██████████| 196/196 [00:24<00:00,  8.02it/s, accuracy=54.8, loss=1.98]


Epoch [2/50], Loss: 1.7909, Accuracy: 54.78%


Epoch 3/50: 100%|██████████| 196/196 [00:23<00:00,  8.45it/s, accuracy=58.6, loss=1.59]


...

Epoch 49/50: 100%|██████████| 196/196 [00:21<00:00,  9.09it/s, accuracy=72.4, loss=0.836]


Epoch [49/50], Loss: 0.9527, Accuracy: 72.41%


Epoch 50/50: 100%|██████████| 196/196 [00:22<00:00,  8.59it/s, accuracy=72.5, loss=0.927]

Epoch [50/50], Loss: 0.9474, Accuracy: 72.46%


NameError: name 'evaluate' is not defined

## 2.2 从零开始训练

In [19]:
# 加载未经过任何预训练的 ResNet-18 模型
scratch_model = resnet18(weights=None).to(device)

# 替换全连接层，调整输出维度为 100
num_features = scratch_model.fc.in_features
scratch_model.fc = nn.Linear(num_features, 100).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(scratch_model.parameters(), lr=1e-3)

# 从零开始训练模型
train_linear_classifier(scratch_model, train_loader, criterion, optimizer, epochs=50)


Epoch 1/50: 100%|██████████| 196/196 [00:39<00:00,  4.91it/s, accuracy=13.4, loss=3.54]


Epoch [1/50], Loss: 3.6796, Accuracy: 13.39%


Epoch 2/50: 100%|██████████| 196/196 [00:40<00:00,  4.85it/s, accuracy=26.6, loss=2.77]


Epoch [2/50], Loss: 2.9125, Accuracy: 26.60%


Epoch 3/50: 100%|██████████| 196/196 [00:39<00:00,  4.99it/s, accuracy=38.9, loss=2.17]


...

Epoch [48/50], Loss: 0.0038, Accuracy: 99.96%


Epoch 49/50: 100%|██████████| 196/196 [00:36<00:00,  5.34it/s, accuracy=100, loss=0.00219] 


Epoch [49/50], Loss: 0.0026, Accuracy: 99.96%


Epoch 50/50: 100%|██████████| 196/196 [00:38<00:00,  5.05it/s, accuracy=100, loss=0.00182] 

Epoch [50/50], Loss: 0.0018, Accuracy: 99.97%


# 3. 评估有监督与无监督

In [ ]:
def evaluate(model, dataloader, criterion):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100. * correct / total
    test_loss /= len(dataloader)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return test_loss, accuracy

test_loss, test_accuracy = evaluate(linear_classifier, test_loader, criterion)


In [ ]:
test_loss, test_accuracy = evaluate(pretrained_model, test_loader, criterion)

In [ ]:
test_loss, test_accuracy = evaluate(scratch_model, test_loader, criterion)